## Python GA Report Extraction

In [8]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime

In [9]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_Secrets.json'
VIEW_ID = '16182921' # Edit This

In [10]:
'''
Initializes an Analytics Reporting API V4 service object. 
Returns: An authorized Analytics Reporting API V4 service object.
'''

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

In [ ]:
# Operators & Expression
'''
REGEXP, 
BEGINS_WITH, 
ENDS_WITH, 
PARTIAL, 
EXACT, 
NUMERIC_EQUAL, 
NUMERIC_GREATER_THAN, 
NUMERIC_LESS_THAN, 
IN_LIST
'''

# Filter ALternate - Unclear
'''
{
“dimensionName”: string,
“not”: boolean,
“operator”: enum(Operator),
“expressions”: [
string
],
“caseSensitive”: boolean,
}
'''

# AND OR
'''
And -;
OR -,
'''

# Link for GA Dimension names and Metric Names
'''
Dimension and Metrics - https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/
'''

In [ ]:
POST https://analyticsreporting.googleapis.com/v4/reports:batchGet
{
  "reportRequests":[
  {
    ...
    "orderBys": [
    {
      "fieldName": "ga:users",
      "sortOrder": "DESCENDING"
    },{
      "fieldName": "ga:source"
    }],
  }]
}

In [85]:
"""
Queries the Analytics Reporting API V4.
Args:
analytics: An authorized Analytics Reporting API V4 service object.
Returns:
The Analytics Reporting API V4 response.
"""

def get_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                  'viewId': "16182921",#Edit This
                  'dateRanges': [{'startDate': '2019-04-01', 'endDate': '2020-02-15'}],
                  'metrics': [{'expression': 'ga:Users'},{'expression': 'ga:Sessions'},{'expression': 'ga:Pageviews'}],
                  'dimensions': [{'name': 'ga:pagePath'}],
                  #  'metricFilterClauses': [{
                  #      'filters': [{
                  #          "metricName": "ga:pageviews",
                  #          "operator": "GREATER_THAN",
                  #          "comparisonValue": "1000"
                  #          }]
                  #      }],
                    'dimensionFilterClauses': [
                        {
                            'filters': [
                                {
                                    "dimensionName": "ga:pagePath",
                                    "operator": "REGEXP",
                                    "expressions": ["articleshow"]
                                    }
                                ]
                            }
                        ],
                    'orderBys': [
                        {
                            "fieldName": "ga:users",
                            "sortOrder": "DESCENDING"
                            }
                    ],
                    'pageSize': 1000000
                    }
                ]
            }
        ).execute()

In [58]:
'''def get_report(analytics):
    return analytics.reports().batchGet(
  body={
    'reportRequests': [
    {
      'viewId': VIEW_ID,
      'dateRanges': [{'startDate': '2016-07-01', 'endDate': 'today'}],
      'metrics': [{'expression': 'ga:pageviews'}],
      'dimensions': [{'name': 'ga:country'}, {'name': 'ga:city'}],
      'metricFilterClauses': [{
      'filters': [{
          "metricName": "ga:pageviews",
          "operator": "GREATER_THAN",
          "comparisonValue": "1000"
      }]
      }],
      'dimensionFilterClauses': [
        {
          'filters': [
            {
              "dimensionName": "ga:country",
              "operator": "EXACT",
              "expressions": ["United States"]
            }
          ]
        }
      ]
    }]
  }
).execute()

'''

'def get_report(analytics):\n    return analytics.reports().batchGet(\n  body={\n    \'reportRequests\': [\n    {\n      \'viewId\': VIEW_ID,\n      \'dateRanges\': [{\'startDate\': \'2016-07-01\', \'endDate\': \'today\'}],\n      \'metrics\': [{\'expression\': \'ga:pageviews\'}],\n      \'dimensions\': [{\'name\': \'ga:country\'}, {\'name\': \'ga:city\'}],\n      \'metricFilterClauses\': [{\n      \'filters\': [{\n          "metricName": "ga:pageviews",\n          "operator": "GREATER_THAN",\n          "comparisonValue": "1000"\n      }]\n      }],\n      \'dimensionFilterClauses\': [\n        {\n          \'filters\': [\n            {\n              "dimensionName": "ga:country",\n              "operator": "EXACT",\n              "expressions": ["United States"]\n            }\n          ]\n        }\n      ]\n    }]\n  }\n).execute()\n\n'

In [86]:
#,{'name': 'ga:deviceCategory'},
#{'name': 'ga:browser'},
#{'name': 'ga:dateHour'},
#{'name': 'ga:sourceMedium'},
#{'name': 'ga:screenResolution'}

def print_response(response):
    list1 = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or ',' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = float(value)

        list1.append(dict)
    
    df = pd.DataFrame(list1)
    fn=datetime.datetime.today().strftime('%Y-%m-%d')
    df.to_csv(str(fn)+".csv", sep='\t')
    print(df)
    return df

In [87]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)
df = print_response(response)

       ga:Pageviews  ga:Sessions     ga:Users  \
0       848284690.0  377196607.0  176251609.0   
1               1.0          1.0          1.0   
2               1.0          1.0          1.0   
3               1.0          1.0          1.0   
4               1.0          1.0          1.0   
5               1.0          1.0          1.0   
6               1.0          1.0          1.0   
7               1.0          1.0          1.0   
8               1.0          1.0          1.0   
9               1.0          1.0          1.0   
10              1.0          1.0          1.0   
11              1.0          1.0          1.0   
12              1.0          1.0          1.0   
13              1.0          1.0          1.0   
14              1.0          1.0          1.0   
15              1.0          1.0          1.0   
16              1.0          1.0          1.0   
17              1.0          1.0          1.0   
18              1.0          1.0          1.0   
19              1.0 

In [89]:
df["page"]=df["ga:pagePath"]

In [97]:
df["x1"]=df['page'].str.find(".cms")-8
df["x2"]=df['page'].str.find(".cms")-1

df["msid"]=df["page"].str.slice(start=df["x1"],stop=df["x2"])

In [106]:
df["page"].str[df["x1"]:df["x2"]:1]
#.slice(start=1,stop=8)

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
         ..
99970   NaN
99971   NaN
99972   NaN
99973   NaN
99974   NaN
99975   NaN
99976   NaN
99977   NaN
99978   NaN
99979   NaN
99980   NaN
99981   NaN
99982   NaN
99983   NaN
99984   NaN
99985   NaN
99986   NaN
99987   NaN
99988   NaN
99989   NaN
99990   NaN
99991   NaN
99992   NaN
99993   NaN
99994   NaN
99995   NaN
99996   NaN
99997   NaN
99998   NaN
99999   NaN
Name: page, Length: 100000, dtype: float64

## ------------------------------------------------------------------------------------------------------------------------------------

## Attempt -2

In [3]:
import pandas as pd
import pygsheets
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import datetime

In [4]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = '16182921'

In [7]:
DIMENSIONS=['ga:source','ga:medium']
METRICS=['ga:users','ga:sessions']